In [ ]:
from pynq import Overlay
ps_gpio_design = Overlay("./ps_gpio_kv260.bit")
from pynq import GPIO
from time import sleep

In [ ]:
#Emergency Call
EC = GPIO(GPIO.get_gpio_pin(0), 'in')

#Floor Call request
FR1 = GPIO(GPIO.get_gpio_pin(1), 'in')
FR0 = GPIO(GPIO.get_gpio_pin(2), 'in')

#Motor control pins
EUD = GPIO(GPIO.get_gpio_pin(3), 'out') #Elevator Up(1)/down(0)
ESG = GPIO(GPIO.get_gpio_pin(4), 'out') #Elevator stop(1)/go(0)
EOC = GPIO(GPIO.get_gpio_pin(5), 'out') #Elevator Door open(1)/close(0)

#Current floor
CF1=    GPIO(GPIO.get_gpio_pin(6), 'out') 
CF0=    GPIO(GPIO.get_gpio_pin(7), 'out')

FLOOR_1 = 0
FLOOR_2 = 1
FLOOR_3 = 2
FLOOR_4 = 3

IDLE = 'Idle'
MOVING_UP = 'Moving Up'
MOVING_DOWN = 'Moving Down'
ARRIVING = 'Arriving'
EMERGENCY = 'Emergency'

In [ ]:
current_floor = FLOOR_1
target_floors = set()
state = IDLE
door_open = False
emergency_mode = False

def check_inputs():
    # Check for emergency request
    if EC.read() == 1:
        emergency_call()
        return

    # Process floor requests
    floor_request = (FR1.read() << 1) | FR0.read()  # 2-bit floor call
    if floor_request != current_floor:
        request_floor(floor_request)
    
    # Handle elevator movement and other actions
    check_for_requests()
def request_floor(floor):
    if floor not in target_floors:
        target_floors.add(floor)
    print(f"Floor {floor + 1} requested.")

def emergency_call():
    print("Emergency call received! Moving to Floor 1 immediately.")
    target_floors = {FLOOR_1}  # Clear all requests except for Floor 1
    emergency_mode = True
    
def move_to_floor(target_floor):
    if current_floor == target_floor:
        return
        
    print(f"Moving from Floor {current_floor + 1} to Floor {target_floor + 1}...")
        
    # Simulate the movement and apply acceleration/deceleration
    total_counts = abs(current_floor - target_floor) * 100
    for i in range(total_counts):
        time.sleep(0.05)  # Simulate time taken for each step
        
    # Update position when done
    current_floor = target_floor
    print(f"Arrived at Floor {current_floor + 1}.")
    state = ARRIVING
    open_door()

def open_door():
    print("Opening doors...")
    door_open = True
    time.sleep(2)  # Door remains open for 2 seconds
    close_door()
        
def close_door():
    print("Closing doors...")
    door_open = False
    time.sleep(1)  # Door takes 1 second to close
    state = IDLE
    check_for_requests()
        
def check_for_requests():
    if emergency_mode:
        emergency_mode = False
        print("Elevator is in emergency mode. Returning to Idle.")
        self.state = IDLE
        return
        
    # Sort the target floors and determine next floor to move to
    if target_floors:
        requested_floors = sorted(target_floors)
        next_floor = requested_floors.pop(0)  # Get the next request in sorted order
        target_floors = set(requested_floors)  # Update remaining requests
        if next_floor > current_floor:
            state = MOVING_UP
            move_to_floor(next_floor)
        elif next_floor < current_floor:
            state = MOVING_DOWN
            move_to_floor(next_floor)
    else:
        print("No pending requests. Elevator is idle.")
        state = IDLE

def update_motor_output():
    # Pin 3 (Up/Down)
    if state == MOVING_UP:
        EUD.write(1)  # Up
    elif state == MOVING_DOWN:
        EUD.write(0)  # Down
    else:
        EUD.write(0)  # Stop

    # Pin 4 (Stop/Go)
    if state == MOVING_UP or state == MOVING_DOWN:
        ESG.write(0)  # Go
    else:
        ESG.write(1)  # Stop

    # Pin 5 (Door Open/Close)
    if door_open:
        ECO.write(1)  # Door Open
    else:
        ECO.write(0)  # Door Close

def update_floor_output():
    if current_floor == FLOOR_1:
        CF1.write(0)
        CF0.write(0)
    elif current_floor == FLOOR_2:
        CF1.write(0)
        CF0.write(1)
    elif self.current_floor == FLOOR_3:
        CF1.write(1)
        CF0.write(0)
    elif self.current_floor == FLOOR_4:
        CF1.write(1)
        CF0.write(1)
